-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Extract and Load Data Lab

In this lab, you will extract and load raw data from JSON files into a Delta table.

## Learning Objectives
By the end of this lab, you should be able to:
- Create an external table to extract data from JSON files
- Create an empty Delta table with a provided schema
- Insert records from an existing table into a Delta table
- Use a CTAS statement to create a Delta table from files

## Run Setup

Run the following cell to configure variables and datasets for this lesson.

In [0]:
%run ../Includes/Classroom-Setup-4.5L

Python interpreter will be restarted.
Python interpreter will be restarted.

Creating the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_5l"
Skipping install to "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss", dataset already exists

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.5l
 DA.paths.user_db: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.5l/4_5l.db
 DA.paths.datasets: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss

Predefined tables in dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_5l:
 -none-

Setup completed in 1 seconds

## Overview of the Data

We will work with a sample of raw Kafka data written as JSON files. 

Each file contains all records consumed during a 5-second interval, stored with the full Kafka schema as a multiple-record JSON file. 

The schema for the table:

| field  | type | description |
| ------ | ---- | ----------- |
| key    | BINARY | The **`user_id`** field is used as the key; this is a unique alphanumeric field that corresponds to session/cookie information |
| offset | LONG | This is a unique value, monotonically increasing for each partition |
| partition | INTEGER | Our current Kafka implementation uses only 2 partitions (0 and 1) |
| timestamp | LONG    | This timestamp is recorded as milliseconds since epoch, and represents the time at which the producer appends a record to a partition |
| topic | STRING | While the Kafka service hosts multiple topics, only those records from the **`clickstream`** topic are included here |
| value | BINARY | This is the full data payload (to be discussed later), sent as JSON |

## Extract Raw Events From JSON Files
To load this data into Delta properly, we first need to extract the JSON data using the correct schema.

Create an external table against JSON files located at the filepath provided below. Name this table **`events_json`** and declare the schema above.

In [0]:
%sql
-- TODO
create table if not exists events_json 
(key binary,offset bigint,partition int, timestamp long,topic string,value binary)
using json
options(path='${da.paths.datasets}/raw/events-kafka/')

In [0]:
%sql
describe history events_json

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: DESCRIBE HISTORY is only supported for Delta tables.
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.notADeltaTableException(DeltaErrors.scala:438)
	at com.databricks.sql.transaction.tahoe.commands.DescribeDeltaHistoryCommand.$anonfun$run$1(DescribeDeltaHistoryCommand.scala:79)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:1585)
	at com.databricks.sql.transaction.tahoe.commands.DescribeDeltaHistoryCommand.run(DescribeDeltaHistoryCommand.scala:62)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:80)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult(commands.scala:78)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.executeCollect(commands.scala:89)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:68)
	at org.apache.spark.sql.execution.collect.Collector$.collect(Collector.scala:87)
	at org.apache.spark.sql.execution.collect.InternalRowFormat$.collect(cachedSparkResults.scala:75)
	at org.apache.spark.sql.execution.collect.InternalRowFormat$.collect(cachedSparkResults.scala:62)
	at org.apache.spark.sql.execution.ResultCacheManager.collectResult$1(ResultCacheManager.scala:589)
	at org.apache.spark.sql.execution.ResultCacheManager.computeResult(ResultCacheManager.scala:596)
	at org.apache.spark.sql.execution.ResultCacheManager.$anonfun$getOrComputeResultInternal$1(ResultCacheManager.scala:542)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.sql.execution.ResultCacheManager.getOrComputeResultInternal(ResultCacheManager.scala:541)
	at org.apache.spark.sql.execution.ResultCacheManager.getOrComputeResult(ResultCacheManager.scala:438)
	at org.apache.spark.sql.execution.ResultCacheManager.getOrComputeResult(ResultCacheManager.scala:417)
	at org.apache.spark.sql.execution.SparkPlan.executeCollectResult(SparkPlan.scala:422)
	at org.apache.spark.sql.Dataset.collectResult(Dataset.scala:3153)
	at org.apache.spark.sql.Dataset.$anonfun$collectResult$1(Dataset.scala:3144)
	at org.apache.spark.sql.Dataset.$anonfun$withAction$1(Dataset.scala:3951)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$8(SQLExecution.scala:239)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:386)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:186)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:968)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:141)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:336)
	at org.apache.spark.sql.Dataset.withAction(Dataset.scala:3949)
	at org.apache.spark.sql.Dataset.collectResult(Dataset.scala:3143)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation0(OutputAggregator.scala:271)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation(OutputAggregator.scala:105)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:115)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:145)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:605)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:33)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:31)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:205)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scal

**NOTE**: We'll use Python to run checks occasionally throughout the lab. The following cell will return an error with a message on what needs to change if you have not followed instructions. No output from cell execution means that you have completed this step.

In [0]:
%python
assert spark.table("events_json"), "Table named `events_json` does not exist"
assert spark.table("events_json").columns == ['key', 'offset', 'partition', 'timestamp', 'topic', 'value'], "Please name the columns in the order provided above"
assert spark.table("events_json").dtypes == [('key', 'binary'), ('offset', 'bigint'), ('partition', 'int'), ('timestamp', 'bigint'), ('topic', 'string'), ('value', 'binary')], "Please make sure the column types are identical to those provided above"

total = spark.table("events_json").count()
assert total == 2252, f"Expected 2252 records, found {total}"

## Insert Raw Events Into Delta Table
Create an empty managed Delta table named **`events_raw`** using the same schema.

In [0]:
%sql
-- TODO
create or replace table events_raw
(key binary,offset bigint,partition int, timestamp long,topic string,value binary)

In [0]:
%sql
describe history events_raw

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2022-07-29T06:54:49.000+0000,6997591375752473,manujkumar.joshi@celebaltech.com,CREATE OR REPLACE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(2331746562401938),0725-045645-b5m629fz,null,WriteSerializable,true,Map(),null,Databricks-Runtime/10.4.x-scala2.12


Run the cell below to confirm the table was created correctly.

In [0]:
%python
assert spark.table("events_raw"), "Table named `events_raw` does not exist"
assert spark.table("events_raw").columns == ['key', 'offset', 'partition', 'timestamp', 'topic', 'value'], "Please name the columns in the order provided above"
assert spark.table("events_raw").dtypes == [('key', 'binary'), ('offset', 'bigint'), ('partition', 'int'), ('timestamp', 'bigint'), ('topic', 'string'), ('value', 'binary')], "Please make sure the column types are identical to those provided above"
assert spark.table("events_raw").count() == 0, "The table should have 0 records"

### Once the extracted data and Delta table are ready, insert the JSON records from the **`events_json`** table into the new **`events_raw`** Delta table.

In [0]:
%sql
-- TODO
insert into events_raw
select * from events_json

num_affected_rows,num_inserted_rows
2252,2252


Manually review the table contents to ensure data was written as expected.

In [0]:
%sql
-- TODO
select * from events_raw

key,offset,partition,timestamp,topic,value
VUEwMDAwMDAxMDczOTgwNTQ=,219255030,0,1593880885085,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6Im1haW4iLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODUwMzYxMjksImdlbyI6eyJjaXR5IjoiTmV3IFlvcmsiLCJzdGF0ZSI6Ik5ZIn0= (truncated)
VUEwMDAwMDAxMDczOTI0NTg=,219255043,0,1593880892303,clickstream,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoiYWRkX2l0ZW0iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzMDA2OTY3NTEsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg5MjI= (truncated)
VUEwMDAwMDAxMDczOTU5Njg=,219255108,0,1593880889174,clickstream,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJwcmVtaXVtIiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODYxMDMwMjQxLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA4ODk= (truncated)
VUEwMDAwMDAxMDczOTgwMzA=,219255118,0,1593880889725,clickstream,eyJkZXZpY2UiOiJpT1MiLCJlY29tbWVyY2UiOnt9LCJldmVudF9uYW1lIjoib3JpZ2luYWwiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODA4ODI0Mjk5ODAsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4OTY= (truncated)
VUEwMDAwMDAxMDczODIyMzM=,219438025,1,1593880886106,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg= (truncated)
VUEwMDAwMDAxMDczODIyMzM=,219438069,1,1593880886106,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNjX2luZm8iLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4ODAzNjQzMjEwODgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg= (truncated)
VUEwMDAwMDAxMDczOTgwMzc=,219438089,1,1593880887640,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImRlbGl2ZXJ5IiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODgyOTY0MjYyLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA= (truncated)
VUEwMDAwMDAxMDczOTgxNTk=,219438114,1,1593880894803,clickstream,eyJkZXZpY2UiOiJtYWNPUyIsImVjb21tZXJjZSI6e30sImV2ZW50X25hbWUiOiJtYWluIiwiZXZlbnRfdGltZXN0YW1wIjoxNTkzODgwODk0Nzg5NTc5LCJnZW8iOnsiY2l0eSI6Ikxha2V3b29kIiwic3RhdGUiOiJDTyJ9LCI= (truncated)
VUEwMDAwMDAxMDczNzY0Njc=,219438126,1,1593880888445,clickstream,eyJkZXZpY2UiOiJXaW5kb3dzIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImNhcnQiLCJldmVudF9wcmV2aW91c190aW1lc3RhbXAiOjE1OTM4Nzk2MTk4NTI2NzgsImV2ZW50X3RpbWVzdGFtcCI6MTU5Mzg4MDg4ODM= (truncated)
VUEwMDAwMDAxMDczOTgwMzc=,219438135,1,1593880887640,clickstream,eyJkZXZpY2UiOiJBbmRyb2lkIiwiZWNvbW1lcmNlIjp7fSwiZXZlbnRfbmFtZSI6ImRlbGl2ZXJ5IiwiZXZlbnRfcHJldmlvdXNfdGltZXN0YW1wIjoxNTkzODgwODgyOTY0MjYyLCJldmVudF90aW1lc3RhbXAiOjE1OTM4ODA= (truncated)


Run the cell below to confirm the data has been loaded correctly.

In [0]:
%python
assert spark.table("events_raw").count() == 2252, "The table should have 2252 records"
assert set(row['timestamp'] for row in spark.table("events_raw").select("timestamp").limit(5).collect()) == {1593880885085, 1593880892303, 1593880889174, 1593880886106, 1593880889725}, "Make sure you have not modified the data provided"

## Create Delta Table from a Query
In addition to new events data, let's also load a small lookup table that provides product details that we'll use later in the course.
Use a CTAS statement to create a managed Delta table named **`item_lookup`** that extracts data from the parquet directory provided below.

In [0]:
%sql
-- TODO
create or replace table item_lookup
as select * from parquet.`${da.paths.datasets}/raw/item-lookup`

num_affected_rows,num_inserted_rows


In [0]:
%sql
describe history item_lookup

Run the cell below to confirm the lookup table has been loaded correctly.

In [0]:
%python
assert spark.table("item_lookup").count() == 12, "The table should have 12 records"
assert set(row['item_id'] for row in spark.table("item_lookup").select("item_id").limit(5).collect()) == {'M_PREM_F', 'M_PREM_K', 'M_PREM_Q', 'M_PREM_T', 'M_STAN_F'}, "Make sure you have not modified the data provided"

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python
DA.cleanup()

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>